### About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





### Importing the Dependencies

In [1]:
import numpy as np  #For Making Numpy Array
import pandas as pd  # For Creating DataFrame
import re            #Regular exp. used for searching in paragraph
from nltk.corpus import stopwords   #stopwords used to remove the value that does not useable 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split    #data split into traing and testing data
from sklearn.linear_model import LogisticRegression    #model training
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krmuk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Pre-processing

In [4]:
# loading the dataset to a pandas DataFrame
#https://drive.google.com/file/d/1gld3wVjHb2ogEFWQbnsFpF-wtiVTkaFS/view?usp=sharing

path="https://drive.google.com/uc?export=download&id=1gld3wVjHb2ogEFWQbnsFpF-wtiVTkaFS"
news_dataset=pd.read_csv(path)

In [5]:
news_dataset.shape

(20800, 5)

In [6]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [9]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']
news_dataset.shape

(20800, 6)

In [10]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [11]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [12]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)    # id,auth,title,text everything in X
Y = news_dataset['label']       #label is dependent

In [13]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [14]:
print(Y)  

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


### Stemming:

Stemming is the process of reducing a word to its Root word . for better accuracy

example:
actor, actress, acting --> act

In [15]:
port_stem = PorterStemmer()

In [16]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)  # Removes everything by ' ' except alphabet content is same data combined of both auth. & title
    stemmed_content = stemmed_content.lower()          # Converted into lowercase
    stemmed_content = stemmed_content.split()          # Converted into list
    # steam the word that is also not stopwatch in steameed_word
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)    # Join everything
    return stemmed_content

In [17]:
# Apply steaming on content
news_dataset['content'] = news_dataset['content'].apply(stemming)  

In [18]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [19]:
#separating the data and label
X = news_dataset['content'].values  # X have only auth & title data 
Y = news_dataset['label'].values

In [20]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [21]:
print(Y)

[1 0 1 ... 0 1 1]


In [22]:
Y.shape

(20800,)

In [23]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [24]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

### Splitting The Dataset to Training & Test Data

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

#label for X_train in X_test AND for Y_train in Y_test

In [26]:
print(X_train)

  (0, 16996)	0.09995656818816077
  (0, 16928)	0.3360072514082535
  (0, 15295)	0.09807665903342763
  (0, 13914)	0.3334797245354899
  (0, 13014)	0.2680313811122545
  (0, 12501)	0.3929876463935473
  (0, 11936)	0.24142639024498436
  (0, 10306)	0.09662001419895176
  (0, 10219)	0.3019527708144002
  (0, 3155)	0.3400831511004003
  (0, 2794)	0.3776836172783757
  (0, 336)	0.3360072514082535
  (1, 16996)	0.07263181421455335
  (1, 15424)	0.22579404836928033
  (1, 15417)	0.26613170238131584
  (1, 15295)	0.07126580880898774
  (1, 13453)	0.3387500815971264
  (1, 11421)	0.3084666283145136
  (1, 10306)	0.07020736153621741
  (1, 10061)	0.24924889065491332
  (1, 9856)	0.3387500815971264
  (1, 5313)	0.2768869285533855
  (1, 2761)	0.3387500815971264
  (1, 2207)	0.3387500815971264
  (1, 956)	0.2855580802628186
  :	:
  (16637, 10008)	0.32617532466383187
  (16637, 9588)	0.184164927958451
  (16637, 8153)	0.3214731053599403
  (16637, 5009)	0.3538476463585765
  (16637, 1868)	0.40919228974806576
  (16637, 364)	0.

In [27]:
X_train.shape

(16640, 17128)

In [28]:
print(X_test)


  (0, 12801)	0.2910746804557067
  (0, 9818)	0.30786004182651133
  (0, 7668)	0.22945314906455008
  (0, 6816)	0.16094563145945953
  (0, 6289)	0.288254092437116
  (0, 5941)	0.288254092437116
  (0, 5233)	0.21316265672448448
  (0, 4346)	0.3250084367199054
  (0, 3395)	0.3301936745912874
  (0, 2959)	0.24534646237198773
  (0, 1667)	0.30373060380734146
  (0, 908)	0.213510750423647
  (0, 239)	0.34297808354766485
  (1, 16996)	0.09117761343372983
  (1, 15295)	0.08946281236254729
  (1, 14046)	0.42524648908354634
  (1, 13190)	0.36773046084789346
  (1, 12741)	0.24868518461414146
  (1, 12279)	0.3796661151115819
  (1, 12041)	0.37327055071909065
  (1, 10306)	0.08813410128297053
  (1, 8813)	0.42524648908354634
  (1, 4008)	0.23098933893199997
  (1, 3339)	0.2834482751186189
  (2, 16868)	0.344315415802567
  :	:
  (4158, 16782)	0.2796201559111399
  (4158, 13954)	0.2713157796360236
  (4158, 13918)	0.29432702492466434
  (4158, 13775)	0.21479309786895925
  (4158, 13600)	0.2496550139728973
  (4158, 11068)	0.2943

In [29]:
print(Y_train)

[0 0 0 ... 0 0 1]


In [30]:
print(Y_test)

[1 0 1 ... 1 1 0]


### MODEL PREDICTION

##### 1.Training the Model: Logistic Regression

In [31]:
model = LogisticRegression()

In [32]:
model.fit(X_train, Y_train)

LogisticRegression()

#### Evaluation

accuracy score

In [33]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [34]:
print(X_train_prediction)

[0 0 0 ... 0 0 1]


In [35]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9865985576923076


In [36]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [37]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9790865384615385


Accuracy is 97 % which is very good score

#### 2. SVM-(Support vector Classifier)

In [38]:
from sklearn.svm import SVC
# Applied SVC
classification_rbf = SVC(kernel="rbf")    
#Train the Model USing SVC
classification_rbf.fit(X_train,Y_train)

SVC()

In [39]:
#Test the Model using sVC
classification_rbf.score(X_test,Y_test)

0.9889423076923077

In [40]:
classification_poly = SVC(kernel="poly",degree=2)    

#Train the Model USing SVC
classification_poly.fit(X_train,Y_train)
classification_poly.score(X_test,Y_test)

0.9901442307692307

In [41]:
classification_linear = SVC(kernel="linear")    
#Train the Model USing SVC
classification_linear.fit(X_train,Y_train)

classification_linear.score(X_test,Y_test)

0.9913461538461539

#### 3.Decision Tree Classification

In [42]:
#1.Gini Impurity
#2.Entropy

In [43]:
from sklearn.tree import DecisionTreeClassifier

In [44]:
classifier=DecisionTreeClassifier(criterion="gini")
# Train the model
classifier.fit(X_train,Y_train)

DecisionTreeClassifier()

In [45]:
classifier.score(X_test,Y_test)

0.9923076923076923

In [46]:
# CLASSIFIER ENTROPY
classifier_entropy=DecisionTreeClassifier(criterion="entropy")
# Train the model
classifier_entropy.fit(X_train,Y_train,)

DecisionTreeClassifier(criterion='entropy')

In [47]:
# Check Accuracy
classifier.score(X_test,Y_test)

0.9923076923076923

#### 4.RandomForestClassifier

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
classifier=RandomForestClassifier(n_estimators=100, criterion="gini",)
# Train the model
classifier.fit(X_train,Y_train)

RandomForestClassifier()

In [50]:
classifier.score(X_test,Y_test)

0.9920673076923077

Making a Predictive System

In [51]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [52]:
print(X_test[3])


  (0, 1894)	0.10717190161988106
  (0, 2407)	0.22941655373198824
  (0, 2510)	0.19575583865343715
  (0, 2806)	0.22586908309808348
  (0, 3782)	0.5146474416282095
  (0, 5529)	0.2895160407717208
  (0, 6868)	0.2729431954288567
  (0, 7668)	0.18748490820807126
  (0, 11217)	0.3302474074203597
  (0, 11246)	0.340188979812318
  (0, 11888)	0.25336998543209355
  (0, 12693)	0.211577404470594
  (0, 14632)	0.23337028911399943


In [53]:
#Cross check from Orignal data
print(Y_test[3])

0


In [54]:
X_new2 = X_test[0]   # 1st row news we check

prediction = model.predict(X_new2)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [55]:
print(X_new2[0])   #logistic regression Finding the  all of these value threshold and if thershold>0.8 means it is 1 which is fake news

  (0, 239)	0.34297808354766485
  (0, 908)	0.213510750423647
  (0, 1667)	0.30373060380734146
  (0, 2959)	0.24534646237198773
  (0, 3395)	0.3301936745912874
  (0, 4346)	0.3250084367199054
  (0, 5233)	0.21316265672448448
  (0, 5941)	0.288254092437116
  (0, 6289)	0.288254092437116
  (0, 6816)	0.16094563145945953
  (0, 7668)	0.22945314906455008
  (0, 9818)	0.30786004182651133
  (0, 12801)	0.2910746804557067


In [56]:
print(Y_test[0])

1


In [65]:

def output_lable(n):
    if n == 1:
        return "Fake News"
    elif n == 0:
        return "Real News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(stemming) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorizer.transform(new_x_test)
    pred_LR = classification_rbf.predict(new_xv_test)
    return print("\n\nLR Prediction: {}".format(output_lable(pred_LR[0]))) 

In [66]:
news = str(input())
manual_testing(news)

State Polls: Trump Gains DRAMATICALLY | Daily Wire	nan


LR Prediction: Fake News


In [67]:
# Save FILE
import pickle


In [68]:
filename='finalized_model.pkl'
pickle.dump(model,open(filename,'wb'))

In [69]:
filename='vectorizer.pkl'    #dump tfvectorizer function also
pickle.dump(vectorizer,open(filename,'wb'))